In [1]:
import ipywidgets as widgets
import redis
import json

In [2]:
import subprocess
positionv=[]
def addposition(x,y):
        positionv.append((x,y))
    
def isDetect(line):
    return line.startswith("detection=")
def toPosition(line):
      plotline=line[10:]
      plotv=plotline.split(",")
      return plotv
def apriltag(imagename):
#    imagename='/Users/aadebuger/GEXT/cloud2019/machinevision/data/tz_belt_output20190417_5/40.jpg'
    
#    lines=subprocess.getoutput('/Users/aadebuger/GEXT/github2019/apriltags/xcodebuild/Debug/apriltag_demo {0}'.format(imagename))
    lines=subprocess.getoutput('/home/aadebugergf/Ai/cloud2019/apriltags/build/apriltag_demo {0}'.format(imagename))
    linev=lines.split("\n")
    detectionv=filter(lambda item:isDetect(item),linev)
    positionv=map(lambda item:toPosition(item),detectionv)
    return list(positionv)
import numpy as np 
def getBox(plotv):
            arr2=np.asarray(plotv[1:],float)
            rect=(arr2[6],arr2[7],arr2[4]-arr2[0],arr2[1]-arr2[5])
            return (rect[0]+rect[2]/2,rect[1]+rect[3]/2)

def computeposition(x,y):
    yuandian=(853.5124299065,706.912679109)
    kx = -0.8937383178 
    ky = -0.8729680915
    scarax = yuandian[0] + kx*x 
    scaray = yuandian[1] + ky*y
    return (scarax,scaray)
r = redis.Redis(host='localhost', port=6379, db=0)
import base64
#import cStringIO
import sys
import tempfile
import time
MODEL_BASE = '/Users/aadebuger/GEXT/github2019/models/research'
MODEL_BASE = '/home/aadebugergf/Ai/github2019/models/research'


sys.path.append(MODEL_BASE)
sys.path.append(MODEL_BASE + '/object_detection')
sys.path.append(MODEL_BASE + '/slim')


import numpy as np
from PIL import Image
from PIL import ImageDraw
import tensorflow as tf
from utils import label_map_util




PATH_TO_CKPT = '/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_LABELS = MODEL_BASE + '/object_detection/data/mscoco_label_map.pbtxt'

content_types = {'jpg': 'image/jpeg',
                 'jpeg': 'image/jpeg',
                 'png': 'image/png'}
extensions = sorted(content_types.keys())


def is_image():
  def _is_image(form, field):
    if not field.data:
      raise ValidationError()
    elif field.data.filename.split('.')[-1].lower() not in extensions:
      raise ValidationError()

  return _is_image





class ObjectDetector(object):

  def __init__(self):
    self.detection_graph = self._build_graph()
    self.sess = tf.Session(graph=self.detection_graph)

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    self.category_index = label_map_util.create_category_index(categories)

  def _build_graph(self):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    return detection_graph

  def _load_image_into_numpy_array(self, image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

  def detect(self, image):
    image_np = self._load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    graph = self.detection_graph
    image_tensor = graph.get_tensor_by_name('image_tensor:0')
    boxes = graph.get_tensor_by_name('detection_boxes:0')
    scores = graph.get_tensor_by_name('detection_scores:0')
    classes = graph.get_tensor_by_name('detection_classes:0')
    num_detections = graph.get_tensor_by_name('num_detections:0')

    (boxes, scores, classes, num_detections) = self.sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    boxes, scores, classes, num_detections = map(
        np.squeeze, [boxes, scores, classes, num_detections])

    return boxes, scores, classes.astype(int), num_detections
 

def draw_bounding_box_on_image(image, box, color='red', thickness=4):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  ymin, xmin, ymax, xmax = box
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  if (right-left)<100:
        print("too lower")
        return None
  print("rect,width="+str(right-left)+"height="+str(bottom-top))
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  print('(left, right, top, bottom)',(left, right, top, bottom))
  centerx=left+(right-left)/2
  centery=top+ (bottom-top)/2
  print("center=",centerx,"y=",centery)
  scaraxy=computeposition(centerx*4/3,centery*4/3)
  print("scara xy",scaraxy)
  return scaraxy
    
from io import BytesIO
def encode_image(image):
  image_buffer = BytesIO()
  image.save(image_buffer, format='PNG')
  imagevalue = image_buffer.getvalue()
  base64str=base64.b64encode(imagevalue).decode()

  imgstr = 'data:image/png;base64,{:s}'.format(
     base64str )
  return imgstr


def detect_objects(image_path):
  start = time.time()
  
  image = Image.open(image_path).convert('RGB')
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
  print(num_detections)
  new_images = {}
  for i in range(int(num_detections)):
    
    if scores[i] < 0.8: continue
    print("scres[i",scores[i])
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)

  result = {}
#  result['original'] = encode_image(image.copy())
#  print(new_images)
  count = 0 
  for cls, new_image in new_images.items():
    category = client.category_index[cls]['name']
    print(category)
    result[category] = encode_image(new_image)
    new_image.save("{0}.jpg".format(count))
    count=count+1
  end = time.time()
  print("execute2 time",(end-start)) 
  return result

def detect_objectsbyframe(frame):
  start = time.time()
  
#  image = Image.open(image_path).convert('RGB')
  height, width, channels = frame.shape
  image= Image.frombytes("RGB", (width,height), frame.tostring())
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
#  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
#  print(num_detections)
  new_images = {}
  
  for i in range(int(num_detections)):

    if scores[i] < 0.8: continue
    print("scres[i",scores[i])
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    box=draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)
    if box is not None:
        print("scarxbox",box)
        return (box,new_images[cls])
    return (None,None)
#  result['original'] = encode_image(image.copy())
#  print(new_images)
  return (None,None)


PATH_TO_CKPT='/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'
PATH_TO_CKPT='/home/aadebugergf/Ai/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'


PATH_TO_CKPT='/home/aadebugergf/Ai/models/fastrcnnstbmodelrtx2/frozen_inference_graph.pb'
client = ObjectDetector()

In [3]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
cap = cv2.VideoCapture(1)
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        box,newframe= detect_objectsbyframe(frame)
        if box is not None:
            img_str = cv2.imencode('.jpg', newframe)[1].tostring()
        else:
            
            img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>60):
            break
videooutput()
cap.release()
print("capture end")

error: OpenCV(4.1.0) /io/opencv/modules/imgcodecs/src/grfmt_base.cpp:145: error: (-10:Unknown error code -10) Raw image encoder error: Empty JPEG image (DNL not supported) in function 'throwOnEror'


In [4]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052815.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        detect_objectsbyframe(frame)
        img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>180):
            break
videooutput()
cap.release()
print("capture end")

error: OpenCV(4.1.0) /io/opencv/modules/imgcodecs/src/grfmt_base.cpp:145: error: (-10:Unknown error code -10) Raw image encoder error: Empty JPEG image (DNL not supported) in function 'throwOnEror'


In [5]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052814.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        detect_objectsbyframe(frame)
        img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>180):
            break
videooutput()
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.9963869
boxes [0.60641366 0.         0.8622956  0.04127472]
too lower
scres[i 0.9993274
boxes [0.6020594  0.         0.86020815 0.04290496]
too lower
scres[i 0.999537
boxes [0.59572375 0.         0.8613166  0.04385705]
too lower
scres[i 0.99915075
boxes [0.5950933  0.         0.861603   0.04559236]
too lower
scres[i 0.9933878
boxes [0.593269   0.00107958 0.86134094 0.04822294]
too lower
scres[i 0.99472094
boxes [0.58926463 0.00273427 0.8578713  0.05314437]
too lower
scres[i 0.9995221
boxes [0.58993804 0.00431782 0.855922   0.05685524]
too lower
scres[i 0.99925154
boxes [0.5891236  0.00410322 0.85391855 0.06095213]
too lower
scres[i 0.99891746
boxes [0.58776426 0.00450626 0.85259485 0.06349167]
too lower
scres[i 0.9991074
boxes [0.5899284  0.00463491 0.85608655 0.06638634]
too lower
scres[i 0.9998022
boxes [0.5885217  0.00399757 0.85518956 0.06851315]
too lower
scres[i 0.99987507
boxes [0.5928039  0.00545951 0.84992474 0.07133022]
too lower
scres[i 0.9998704
boxes [0.5920107  

scres[i 0.9999999
boxes [0.58436966 0.05908336 0.88465273 0.27548164]
rect,width=103.87117266654968height=108.10190677642822
(left, right, top, bottom) (28.360015153884888, 132.23118782043457, 210.37307739257812, 318.47498416900635)
center= 80.29560148715973 y= 264.42403078079224
scara xy (757.8280888400022, 399.13435711240606)
scarxbox (757.8280888400022, 399.13435711240606)
scres[i 0.9999999
boxes [0.584226   0.06357374 0.88595164 0.27776268]
rect,width=102.81069159507751height=108.62122535705566
(left, right, top, bottom) (30.515395402908325, 133.32608699798584, 210.321364402771, 318.94258975982666)
center= 81.92074120044708 y= 264.6319770812988
scara xy (755.891489328611, 398.89231646562246)
scarxbox (755.891489328611, 398.89231646562246)
scres[i 0.9999999
boxes [0.5875679  0.06489983 0.8822293  0.2847166 ]
rect,width=105.51205158233643height=106.07810497283936
(left, right, top, bottom) (31.1519193649292, 136.66397094726562, 211.52445316314697, 317.6025581359863)
center= 83.907945

scres[i 0.9999999
boxes [0.5843275  0.14968255 0.88357127 0.37061617]
rect,width=106.04813575744629height=107.72774934768677
(left, right, top, bottom) (71.84762477874756, 177.89576053619385, 210.3579068183899, 318.08565616607666)
center= 124.8716926574707 y= 264.2217814922333
scara xy (704.7089411911313, 399.36976667966013)
scarxbox (704.7089411911313, 399.36976667966013)
scres[i 0.9999999
boxes [0.58286035 0.15457347 0.88425326 0.37770203]
rect,width=107.1017074584961height=108.50144863128662
(left, right, top, bottom) (74.1952657699585, 181.2969732284546, 209.82972621917725, 318.33117485046387)
center= 127.74611949920654 y= 264.08045053482056
scara xy (701.2836273109018, 399.53426990121005)
scarxbox (701.2836273109018, 399.53426990121005)
scres[i 0.9999999
boxes [0.5834944  0.1576709  0.88270533 0.37985346]
rect,width=106.64762735366821height=107.71592617034912
(left, right, top, bottom) (75.68203210830688, 182.3296594619751, 210.05799293518066, 317.7739191055298)
center= 129.005845

scres[i 0.9999999
boxes [0.5818808  0.24041697 0.88178957 0.4608208 ]
rect,width=105.79383373260498height=107.96715259552002
(left, right, top, bottom) (115.40014743804932, 221.1939811706543, 209.4770908355713, 317.4442434310913)
center= 168.2970643043518 y= 263.4606671333313
scara xy (652.9610497171002, 400.25567141206534)
scarxbox (652.9610497171002, 400.25567141206534)
scres[i 0.9999999
boxes [0.5833662  0.24174222 0.88020366 0.46487144]
rect,width=107.10202217102051height=106.86148166656494
(left, right, top, bottom) (116.03626728057861, 223.13828945159912, 210.0118374824524, 316.87331914901733)
center= 169.58727836608887 y= 263.44257831573486
scara xy (651.4235647235819, 400.2767260261649)
scarxbox (651.4235647235819, 400.2767260261649)
scres[i 0.9999999
boxes [0.5838424  0.24566427 0.8812283  0.46912116]
rect,width=107.25930690765381height=107.05893516540527
(left, right, top, bottom) (117.91884899139404, 225.17815589904785, 210.1832628250122, 317.2421979904175)
center= 171.54850

scres[i 0.9999999
boxes [0.5833569  0.35191703 0.87951875 0.5756653 ]
rect,width=107.39916801452637height=106.61825895309448
(left, right, top, bottom) (168.92017364501953, 276.3193416595459, 210.0084900856018, 316.6267490386963)
center= 222.61975765228271 y= 263.31761956214905
scara xy (588.2273529555735, 400.4221726990103)
scarxbox (588.2273529555735, 400.4221726990103)
scres[i 1.0
boxes [0.582991   0.3567703  0.8804107  0.57943356]
rect,width=106.87836170196533height=107.0710802078247
(left, right, top, bottom) (171.24974727630615, 278.1281089782715, 209.8767614364624, 316.9478416442871)
center= 224.68892812728882 y= 263.41230154037476
scara xy (585.761623702809, 400.31196690456875)
scarxbox (585.761623702809, 400.31196690456875)
scres[i 1.0
boxes [0.57862014 0.3617044  0.88033706 0.5856684 ]
rect,width=107.5027084350586height=108.61809253692627
(left, right, top, bottom) (173.61811637878418, 281.1208248138428, 208.3032488822937, 316.92134141921997)
center= 227.36947059631348 y= 262

scres[i 1.0
boxes [0.5777638  0.4573303  0.8784322  0.67765176]
rect,width=105.75430870056152height=108.24063062667847
(left, right, top, bottom) (219.5185375213623, 325.2728462219238, 207.994966506958, 316.2355971336365)
center= 272.39569187164307 y= 262.11528182029724
scara xy (528.9118066673941, 401.8216426774673)
scarxbox (528.9118066673941, 401.8216426774673)
scres[i 1.0
boxes [0.5767534  0.4596821  0.87760895 0.6816573 ]
rect,width=106.54809951782227height=108.30800771713257
(left, right, top, bottom) (220.64741134643555, 327.1955108642578, 207.63121604919434, 315.9392237663269)
center= 273.9214611053467 y= 261.7852199077606
scara xy (527.0936220963524, 402.2058207012859)
scarxbox (527.0936220963524, 402.2058207012859)
scres[i 0.9999999
boxes [0.5773982  0.4644636  0.87740356 0.6868619 ]
rect,width=106.75117492675781height=108.00193548202515
(left, right, top, bottom) (222.94252395629883, 329.69369888305664, 207.8633451461792, 315.86528062820435)
center= 276.31811141967773 y= 261

In [6]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052813.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        detect_objectsbyframe(frame)
        img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>180):
            break
videooutput()
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.99997556
boxes [0.62806284 0.00786631 0.88649434 0.10697931]
too lower
scres[i 0.9999821
boxes [0.6285084  0.00767667 0.8868408  0.11132   ]
too lower
scres[i 0.9999844
boxes [0.6261426  0.00513083 0.8936841  0.1160173 ]
too lower
scres[i 0.9999927
boxes [0.6273956  0.01101914 0.8969798  0.11702677]
too lower
scres[i 0.99999344
boxes [0.6272149  0.01185775 0.90011495 0.11877678]
too lower
scres[i 0.99999785
boxes [0.6266148  0.01207814 0.9033665  0.12236971]
too lower
scres[i 0.99999845
boxes [0.6289497  0.00912074 0.89592075 0.12807725]
too lower
scres[i 0.999998
boxes [0.6260546  0.01142596 0.9060882  0.1286733 ]
too lower
scres[i 0.9999956
boxes [0.6261082  0.01252604 0.90971    0.13501534]
too lower
scres[i 0.9999994
boxes [0.6240617  0.01089791 0.9054867  0.14030579]
too lower
scres[i 0.9999995
boxes [0.62518513 0.01173724 0.9039429  0.14217003]
too lower
scres[i 0.9999994
boxes [0.6217105  0.01339013 0.9047108  0.14363353]
too lower
scres[i 0.99999905
boxes [0.61452246 

scres[i 0.99999976
boxes [0.615871   0.07957453 0.9123507  0.2935128 ]
rect,width=102.69036412239075height=106.73269271850586
(left, right, top, bottom) (38.195775747299194, 140.88613986968994, 221.71356439590454, 328.4462571144104)
center= 89.54095780849457 y= 275.07991075515747
scara xy (746.8108498985471, 386.7313661731065)
scarxbox (746.8108498985471, 386.7313661731065)
scres[i 0.99999976
boxes [0.6164318  0.08289907 0.91225743 0.29773486]
rect,width=103.1211769580841height=106.49723768234253
(left, right, top, bottom) (39.79155421257019, 142.9127311706543, 221.91543817520142, 328.41267585754395)
center= 91.35214269161224 y= 275.1640570163727
scara xy (744.6525494576638, 386.6334235050266)
scarxbox (744.6525494576638, 386.6334235050266)
scres[i 0.99999976
boxes [0.6194265  0.08838382 0.9124285  0.30245775]
rect,width=102.75548815727234height=105.48072338104248
(left, right, top, bottom) (42.42423176765442, 145.17971992492676, 222.9935359954834, 328.4742593765259)
center= 93.8019758

scres[i 0.9999999
boxes [0.6164772  0.17338149 0.9124079  0.39301467]
rect,width=105.42392492294312height=106.53504610061646
(left, right, top, bottom) (83.22311639785767, 188.64704132080078, 221.9317889213562, 328.46683502197266)
center= 135.93507885932922 y= 275.19931197166443
scara xy (691.5252449601703, 386.59238823697734)
scarxbox (691.5252449601703, 386.59238823697734)
scres[i 0.9999999
boxes [0.61670434 0.18007247 0.91027397 0.39758185]
rect,width=104.40449953079224height=105.68506479263306
(left, right, top, bottom) (86.43478631973267, 190.8392858505249, 222.01356410980225, 327.6986289024353)
center= 138.63703608512878 y= 274.8560965061188
scara xy (688.305454685832, 386.9918764368849)
scarxbox (688.305454685832, 386.9918764368849)
scres[i 0.9999999
boxes [0.6175167  0.18328312 0.91031843 0.40116268]
rect,width=104.58219051361084height=105.40862560272217
(left, right, top, bottom) (87.97589778900146, 192.5580883026123, 222.306010723114, 327.7146363258362)
center= 140.2669930458

scres[i 0.9999999
boxes [0.6123027 0.2748409 0.9093406 0.4932982]
rect,width=104.85950946807861height=106.93364381790161
(left, right, top, bottom) (131.92362785339355, 236.78313732147217, 220.42897939682007, 327.3626232147217)
center= 184.35338258743286 y= 273.8958013057709
scara xy (633.8275205272572, 388.1096191946507)
scarxbox (633.8275205272572, 388.1096191946507)
scres[i 0.9999999
boxes [0.61059743 0.27890933 0.90850985 0.49752513]
rect,width=104.93558406829834height=107.24847078323364
(left, right, top, bottom) (133.8764762878418, 238.81206035614014, 219.81507539749146, 327.0635461807251)
center= 186.34426832199097 y= 273.4393107891083
scara xy (631.4550793708092, 388.64095473480904)
scarxbox (631.4550793708092, 388.64095473480904)
scres[i 0.9999999
boxes [0.6107372  0.28340086 0.90793604 0.5020848 ]
rect,width=104.96828556060791height=106.99157953262329
(left, right, top, bottom) (136.03241443634033, 241.00069999694824, 219.86539363861084, 326.85697317123413)
center= 188.516557

scres[i 0.9999999
boxes [0.60839534 0.3756616  0.9059828  0.5944869 ]
rect,width=105.03613471984863height=107.13148355484009
(left, right, top, bottom) (180.31757354736328, 285.3537082672119, 219.0223217010498, 326.1538052558899)
center= 232.8356409072876 y= 272.58806347846985
scara xy (576.0535845353479, 389.6317703883591)
scarxbox (576.0535845353479, 389.6317703883591)
scres[i 0.9999999
boxes [0.60501474 0.37769184 0.906122   0.5994117 ]
rect,width=106.42554759979248height=108.3986234664917
(left, right, top, bottom) (181.29208087921143, 287.7176284790039, 217.8053069114685, 326.2039303779602)
center= 234.50485467910767 y= 272.00461864471436
scara xy (574.0644641240478, 390.3108753523845)
scarxbox (574.0644641240478, 390.3108753523845)
scres[i 0.9999999
boxes [0.60755914 0.38416877 0.9062727  0.6039261 ]
rect,width=105.48352718353271height=107.53688335418701
(left, right, top, bottom) (184.40101146697998, 289.8845386505127, 218.72129201889038, 326.2581753730774)
center= 237.142775058

scres[i 1.0
boxes [0.60500383 0.4702909  0.9033531  0.69266105]
rect,width=106.7376708984375height=107.40573406219482
(left, right, top, bottom) (225.73963165283203, 332.47730255126953, 217.8013801574707, 325.2071142196655)
center= 279.1084671020508 y= 271.5042471885681
scara xy (520.9125207444686, 390.893286439202)
scarxbox (520.9125207444686, 390.893286439202)
scres[i 0.9999999
boxes [0.6051024  0.47677824 0.9038156  0.70085937]
rect,width=107.55894184112549height=107.5367546081543
(left, right, top, bottom) (228.85355472564697, 336.41249656677246, 217.83687114715576, 325.37362575531006)
center= 282.6330256462097 y= 271.6052484512329
scara xy (516.7124767121429, 390.77572526652517)
scarxbox (516.7124767121429, 390.77572526652517)
scres[i 0.9999999
boxes [0.6074335  0.48329443 0.9029156  0.70562005]
rect,width=106.71629905700684height=106.3735556602478
(left, right, top, bottom) (231.98132514953613, 338.69762420654297, 218.67605924606323, 325.04961490631104)
center= 285.33947467803955

scres[i 1.0
boxes [0.5999932  0.57690775 0.89739156 0.7975685 ]
rect,width=105.91715812683105height=107.06339836120605
(left, right, top, bottom) (276.91572189331055, 382.8328800201416, 215.99756240844727, 323.0609607696533)
center= 329.8743009567261 y= 269.5292615890503
scara xy (460.4173594764796, 393.1920856519367)
scarxbox (460.4173594764796, 393.1920856519367)
scres[i 0.9999999
boxes [0.5997234  0.58211696 0.897145   0.8007578 ]
rect,width=104.9476146697998height=107.07176685333252
(left, right, top, bottom) (279.41614151000977, 384.36375617980957, 215.90042352676392, 322.97219038009644)
center= 331.88994884490967 y= 269.4363069534302
scara xy (458.0154104726631, 393.3002808930745)
scarxbox (458.0154104726631, 393.3002808930745)
scres[i 0.9999999
boxes [0.6018143 0.5885887 0.8983539 0.8039543]
rect,width=103.3754825592041height=106.75423622131348
(left, right, top, bottom) (282.5225830078125, 385.8980655670166, 216.65315866470337, 323.40739488601685)
center= 334.21032428741455 y= 

In [20]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052812.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        detect_objectsbyframe(frame)
        img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>180):
            break
videooutput()
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.96308225
boxes [0.6660953  0.         0.8945452  0.03100802]
too lower
scres[i 0.9900603
boxes [0.6549273 0.        0.8907597 0.0314735]
too lower
scres[i 0.99489117
boxes [0.66769767 0.         0.8938466  0.05035315]
too lower
scres[i 0.9953194
boxes [0.65443313 0.00495846 0.9071263  0.05613484]
too lower
scres[i 0.9999989
boxes [0.65690625 0.00573483 0.9148473  0.05289686]
too lower
scres[i 0.9999999
boxes [0.6468654  0.00624344 0.9133051  0.0576272 ]
too lower
scres[i 1.0
boxes [0.64492595 0.00543669 0.91550165 0.06008175]
too lower
scres[i 1.0
boxes [0.6456696  0.00609675 0.919739   0.06129514]
too lower
scres[i 1.0
boxes [0.63479495 0.00879896 0.91698223 0.06490496]
too lower
scres[i 1.0
boxes [0.6440706  0.00576349 0.9076761  0.07714713]
too lower
scres[i 1.0
boxes [0.64530545 0.00499799 0.90276664 0.07814419]
too lower
scres[i 1.0
boxes [0.6390987  0.00560592 0.90852296 0.08120259]
too lower
scres[i 1.0
boxes [0.6406537  0.00606389 0.91364014 0.08387855]
too lower
scre

scres[i 1.0
boxes [0.63397735 0.05424448 0.92884094 0.2715006 ]
rect,width=104.28293287754059height=106.15088939666748
(left, right, top, bottom) (26.037349104881287, 130.32028198242188, 228.23184728622437, 334.38273668289185)
center= 78.17881554365158 y= 281.3072919845581
scara xy (760.3505591177271, 379.48295936394277)
scarxbox (760.3505591177271, 379.48295936394277)
scres[i 1.0
boxes [0.6343579  0.05931631 0.9286529  0.27694428]
rect,width=104.46142494678497height=105.94618320465088
(left, right, top, bottom) (28.471829295158386, 132.93325424194336, 228.36885452270508, 334.31503772735596)
center= 80.70254176855087 y= 281.3419461250305
scara xy (757.3431580099548, 379.44262341878175)
scarxbox (757.3431580099548, 379.44262341878175)
scres[i 1.0
boxes [0.63401234 0.06554805 0.9287135  0.28198352]
rect,width=103.8890290260315height=106.09241724014282
(left, right, top, bottom) (31.46306276321411, 135.3520917892456, 228.2444429397583, 334.3368601799011)
center= 83.40757727622986 y= 281.2

scres[i 1.0
boxes [0.6268024  0.15352774 0.92487055 0.37076014]
rect,width=104.27155494689941height=107.30453968048096
(left, right, top, bottom) (73.69331359863281, 177.96486854553223, 225.6488585472107, 332.95339822769165)
center= 125.82909107208252 y= 279.3011283874512
scara xy (703.5680563264119, 381.8180484394137)
scarxbox (703.5680563264119, 381.8180484394137)
scres[i 1.0
boxes [0.6263098  0.15592545 0.9265317  0.37801278]
rect,width=106.60191535949707height=108.07986974716187
(left, right, top, bottom) (74.84421730041504, 181.4461326599121, 225.47153234481812, 333.55140209198)
center= 128.14517498016357 y= 279.51146721839905
scara xy (700.8080924118892, 381.57322298898583)
scarxbox (700.8080924118892, 381.57322298898583)
scres[i 1.0
boxes [0.6264467  0.16316342 0.9252594  0.38261557]
rect,width=105.33702850341797height=107.57256746292114
(left, right, top, bottom) (78.31844329833984, 183.6554718017578, 225.52082061767578, 333.0933880805969)
center= 130.98695755004883 y= 279.3071

scres[i 1.0
boxes [0.6287726  0.261522   0.9258599  0.47982275]
rect,width=104.78436470031738height=106.95143222808838
(left, right, top, bottom) (125.53055763244629, 230.31492233276367, 226.3581418991089, 333.30957412719727)
center= 177.92273998260498 y= 279.8338580131531
scara xy (641.4906027992731, 381.19797381990105)
scarxbox (641.4906027992731, 381.19797381990105)
scres[i 1.0
boxes [0.62781435 0.2667058  0.9260003  0.48839885]
rect,width=106.41265869140625height=107.34694004058838
(left, right, top, bottom) (128.01878929138184, 234.4314479827881, 226.01316690444946, 333.36010694503784)
center= 181.22511863708496 y= 279.68663692474365
scara xy (637.5553196747483, 381.36933290333747)
scarxbox (637.5553196747483, 381.36933290333747)
scres[i 1.0
boxes [0.62855315 0.2715752  0.92415476 0.49175894]
rect,width=105.68819046020508height=106.41657829284668
(left, right, top, bottom) (130.3561019897461, 236.04429244995117, 226.2791347503662, 332.6957130432129)
center= 183.20019721984863 y= 2

scres[i 1.0
boxes [0.6247631  0.37077263 0.9203498  0.5913926 ]
rect,width=105.89757442474365height=106.4112138748169
(left, right, top, bottom) (177.97086238861084, 283.8684368133545, 224.91470575332642, 331.3259196281433)
center= 230.91964960098267 y= 278.12031269073486
scara xy (578.336777664703, 383.1924676729813)
scarxbox (578.336777664703, 383.1924676729813)
scres[i 1.0
boxes [0.6248113  0.37459078 0.9217266  0.5945412 ]
rect,width=105.57619571685791height=106.88950538635254
(left, right, top, bottom) (179.8035764694214, 285.3797721862793, 224.9320650100708, 331.82157039642334)
center= 232.59167432785034 y= 278.37681770324707
scara xy (576.3443075757555, 382.89390675133734)
scarxbox (576.3443075757555, 382.89390675133734)
scres[i 1.0
boxes [0.62057585 0.37722    0.9199458  0.6005468 ]
rect,width=107.19685077667236height=107.77317523956299
(left, right, top, bottom) (181.06560230255127, 288.26245307922363, 223.4073042869568, 331.1804795265198)
center= 234.66402769088745 y= 277.293

scres[i 1.0
boxes [0.6199253  0.47294816 0.91459984 0.6932077 ]
rect,width=105.7245683670044height=106.08282566070557
(left, right, top, bottom) (227.015118598938, 332.7396869659424, 223.1731152534485, 329.25594091415405)
center= 279.8774027824402 y= 276.21452808380127
scara xy (519.9962177024852, 385.4107198744812)
scarxbox (519.9962177024852, 385.4107198744812)
scres[i 1.0
boxes [0.61849934 0.47589538 0.91737014 0.6975228 ]
rect,width=106.3811731338501height=107.5934886932373
(left, right, top, bottom) (228.4297800064087, 334.8109531402588, 222.65976190567017, 330.25325059890747)
center= 281.62036657333374 y= 276.4565062522888
scara xy (517.9192130005391, 385.12906758124217)
scarxbox (517.9192130005391, 385.12906758124217)
scres[i 1.0
boxes [0.61807793 0.48134345 0.91536295 0.70223916]
rect,width=106.02993965148926height=107.02260732650757
(left, right, top, bottom) (231.04485511779785, 337.0747947692871, 222.50805616378784, 329.5306634902954)
center= 284.0598249435425 y= 276.0193598

In [23]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052811.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
        img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>360):
            break
videooutput()
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.88346297
boxes [0.67016584 0.         0.9123159  0.04386245]
too lower
scres[i 0.99738723
boxes [0.68323934 0.         0.9141095  0.04516052]
too lower
scres[i 0.9994711
boxes [0.692497   0.         0.9215438  0.04746998]
too lower
scres[i 0.99999917
boxes [6.7850786e-01 5.0635335e-05 9.2484152e-01 4.6911553e-02]
too lower
scres[i 0.9999975
boxes [0.6726111  0.0054215  0.9244393  0.05341304]
too lower
scres[i 1.0
boxes [0.67170584 0.00472987 0.93117535 0.05645908]
too lower
scres[i 1.0
boxes [0.66949457 0.00618667 0.9354792  0.05947141]
too lower
scres[i 1.0
boxes [0.6674871  0.00705943 0.9175477  0.07348068]
too lower
scres[i 1.0
boxes [0.661874   0.00566662 0.9200637  0.07650081]
too lower
scres[i 1.0
boxes [0.6626872  0.0066915  0.919136   0.07785568]
too lower
scres[i 1.0
boxes [0.66644293 0.00387954 0.925434   0.08046346]
too lower
scres[i 1.0
boxes [0.66729015 0.00413539 0.92069125 0.08196515]
too lower
scres[i 1.0
boxes [0.66624653 0.00440355 0.92606264 0.0830635 ]
too

scres[i 1.0
boxes [0.6524447  0.05591204 0.94984317 0.27701384]
rect,width=106.12886130809784height=107.0634412765503
(left, right, top, bottom) (26.837781071662903, 132.96664237976074, 234.88009929656982, 341.9435405731201)
center= 79.90221172571182 y= 288.41181993484497
scara xy (758.2968721781838, 371.21359108958234)
scarxbox (758.2968721781838, 371.21359108958234)
scres[i 1.0
boxes [0.651467   0.06359057 0.9493179  0.28180355]
rect,width=104.74222898483276height=107.22630500793457
(left, right, top, bottom) (30.52347421646118, 135.26570320129395, 234.52812910079956, 341.75443410873413)
center= 82.89458870887756 y= 288.14128160476685
scara xy (754.7310028833065, 371.52848619583017)
scarxbox (754.7310028833065, 371.52848619583017)
scres[i 1.0
boxes [0.65038455 0.069677   0.9472365  0.28443792]
rect,width=103.0852460861206height=106.86669588088989
(left, right, top, bottom) (33.4449577331543, 136.5302038192749, 234.13843631744385, 341.00513219833374)
center= 84.9875807762146 y= 287.57

scres[i 1.0
boxes [0.6481568  0.15307225 0.94687676 0.37526557]
rect,width=106.65279150009155height=107.53917932510376
(left, right, top, bottom) (73.47468137741089, 180.12747287750244, 233.33645582199097, 340.8756351470947)
center= 126.80107712745667 y= 287.10604548454285
scara xy (702.4097880836717, 372.73345699599525)
scarxbox (702.4097880836717, 372.73345699599525)
scres[i 1.0
boxes [0.6515698  0.15904559 0.9475573  0.37980244]
rect,width=105.96328496932983height=106.555495262146
(left, right, top, bottom) (76.34188413619995, 182.30516910552979, 234.56512212753296, 341.12061738967896)
center= 129.32352662086487 y= 287.84286975860596
scara xy (699.4039083943729, 371.87582488892895)
scarxbox (699.4039083943729, 371.87582488892895)
scres[i 1.0
boxes [0.65029836 0.16415115 0.9462037  0.3833068 ]
rect,width=105.19471406936646height=106.52592658996582
(left, right, top, bottom) (78.79255056381226, 183.9872646331787, 234.10740852355957, 340.6333351135254)
center= 131.38990759849548 y= 287

...

scres[i 1.0
boxes [0.640613   0.36120808 0.9462302  0.5872414 ]
rect,width=108.49599838256836height=110.02217531204224
(left, right, top, bottom) (173.37987899780273, 281.8758773803711, 230.62068700790405, 340.6428623199463)
center= 227.6278781890869 y= 285.63177466392517
scara xy (582.2594206570361, 374.4494455088336)
scarxbox (582.2594206570361, 374.4494455088336)
scres[i 1.0
boxes [0.6409805  0.36721802 0.9457695  0.59179157]
rect,width=107.7953052520752height=109.7240424156189
(left, right, top, bottom) (176.2646484375, 284.0599536895752, 230.75297355651855, 340.47701597213745)
center= 230.1623010635376 y= 285.614994764328
scara xy (579.2392728751622, 374.46897659806945)
scarxbox (579.2392728751622, 374.46897659806945)
scres[i 1.0
boxes [0.64169914 0.37035522 0.9459063  0.595362  ]
rect,width=108.00325870513916height=109.51457262039185
(left, right, top, bottom) (177.77050495147705, 285.7737636566162, 231.01168870925903, 340.5262613296509)
center= 231.77213430404663 y= 285.76897501

scres[i 1.0
boxes [0.64110804 0.47185057 0.9402484  0.69446975]
rect,width=106.85720443725586height=107.69054174423218
(left, right, top, bottom) (226.4882755279541, 333.34547996520996, 230.79889297485352, 338.4894347190857)
center= 279.91687774658203 y= 284.6441638469696
scara xy (519.949177318422, 375.5989824821969)
scarxbox (519.949177318422, 375.5989824821969)
scres[i 1.0
boxes [0.64030653 0.47578612 0.941596   0.6989998 ]
rect,width=107.14257717132568height=108.4641981124878
(left, right, top, bottom) (228.3773374557495, 335.5199146270752, 230.51035165786743, 338.9745497703552)
center= 281.94862604141235 y= 284.7424507141113
scara xy (517.5280422474691, 375.4845807504259)
scarxbox (517.5280422474691, 375.4845807504259)
scres[i 1.0
boxes [0.63893384 0.48235425 0.94174653 0.70345306]
rect,width=106.12742900848389height=109.01257038116455
(left, right, top, bottom) (231.5300416946411, 337.657470703125, 230.01618146896362, 339.0287518501282)
center= 284.59375619888306 y= 284.522466659

scres[i 1.0
boxes [0.63612455 0.5734205  0.9317311  0.7910211 ]
rect,width=104.44828033447266height=106.41835927963257
(left, right, top, bottom) (275.2418518066406, 379.6901321411133, 229.0048384666443, 335.42319774627686)
center= 327.46599197387695 y= 282.21401810646057
scara xy (463.28722350191185, 378.4275687344089)
scarxbox (463.28722350191185, 378.4275687344089)
scres[i 1.0
boxes [0.6365927  0.5756298  0.9338792  0.79358906]
rect,width=104.62045669555664height=107.02314376831055
(left, right, top, bottom) (276.30228996276855, 380.9227466583252, 229.17336702346802, 336.19651079177856)
center= 328.6125183105469 y= 282.6849389076233
scara xy (461.9209641426468, 377.87943695702404)
scarxbox (461.9209641426468, 377.87943695702404)
scres[i 1.0
boxes [0.63515496 0.57912254 0.9341881  0.798199  ]
rect,width=105.15669822692871height=107.65193939208984
(left, right, top, bottom) (277.97882080078125, 383.13551902770996, 228.65578651428223, 336.30772590637207)
center= 330.5571699142456 y= 28

scres[i 1.0
boxes [0.62926906 0.67437875 0.93003154 0.89061224]
rect,width=103.79207611083984height=108.27449083328247
(left, right, top, bottom) (323.7018013000488, 427.4938774108887, 226.5368628501892, 334.8113536834717)
center= 375.59783935546875 y= 280.67410826683044
scara xy (405.93085508667156, 380.21995827278755)
scarxbox (405.93085508667156, 380.21995827278755)
scres[i 1.0
boxes [0.6301878  0.67713016 0.93067026 0.8960512 ]
rect,width=105.08211135864258height=108.17368268966675
(left, right, top, bottom) (325.0224781036377, 430.1045894622803, 226.86761140823364, 335.0412940979004)
center= 377.563533782959 y= 280.954452753067
scara xy (403.58843317875966, 379.8936492179709)
scarxbox (403.58843317875966, 379.8936492179709)
scres[i 1.0
boxes [0.62747073 0.68093634 0.93203044 0.90206516]
rect,width=106.1418342590332height=109.64149475097656
(left, right, top, bottom) (326.8494415283203, 432.9912757873535, 225.88946342468262, 335.5309581756592)
center= 379.9203586578369 y= 280.71021

scres[i 1.0
boxes [0.6240908  0.77130026 0.9216315  0.9862586 ]
rect,width=103.18001747131348height=107.11466073989868
(left, right, top, bottom) (370.22412300109863, 473.4041404724121, 224.67268466949463, 331.7873454093933)
center= 421.81413173675537 y= 278.23001503944397
scara xy (350.85715987626617, 383.0647788063337)
scarxbox (350.85715987626617, 383.0647788063337)
scres[i 1.0
boxes [0.62376505 0.7753827  0.92096215 0.98730344]
rect,width=101.72195434570312height=106.99095726013184
(left, right, top, bottom) (372.18369483947754, 473.90564918518066, 224.55541849136353, 331.54637575149536)
center= 423.0446720123291 y= 278.05089712142944
scara xy (349.3907852150976, 383.27326444239054)
scarxbox (349.3907852150976, 383.27326444239054)
scres[i 1.0
boxes [0.6240842 0.7772751 0.9194793 0.9872815]
rect,width=100.80307960510254height=106.34224891662598
(left, right, top, bottom) (373.092041015625, 473.89512062072754, 224.6703028678894, 331.0125517845154)
center= 423.49358081817627 y= 277.84

In [37]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052810.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>360):
            break
videooutput()
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.9857312
boxes [0.7029951  0.         0.94227284 0.03040177]
too lower
scres[i 0.9857992
boxes [0.70153004 0.         0.9455697  0.04382139]
too lower
scres[i 0.9939114
boxes [0.70855916 0.00247107 0.9467086  0.05090674]
too lower
scres[i 0.99999225
boxes [0.7034114  0.00124022 0.94725454 0.05019744]
too lower
scres[i 0.99999976
boxes [0.6888531  0.00268293 0.9577429  0.05085928]
too lower
scres[i 1.0
boxes [0.691967   0.00334777 0.9614556  0.05542584]
too lower
scres[i 1.0
boxes [0.68869233 0.00785587 0.96107966 0.05926039]
too lower
scres[i 0.9999999
boxes [0.6869016  0.00834216 0.9638974  0.06221502]
too lower
scres[i 1.0
boxes [0.6878134  0.0074788  0.9434958  0.07701802]
too lower
scres[i 1.0
boxes [0.68555063 0.00751559 0.95012283 0.08003332]
too lower
scres[i 1.0
boxes [0.686462   0.00534265 0.95181704 0.08177464]
too lower
scres[i 1.0
boxes [0.68659866 0.00596088 0.9658111  0.08370178]
too lower
scres[i 1.0
boxes [0.6890269  0.00582077 0.96287847 0.08577896]
too lower


scres[i 1.0
boxes [0.6745813  0.05231627 0.9712272  0.26944938]
rect,width=104.22389209270477height=106.79253816604614
(left, right, top, bottom) (25.111811757087708, 129.33570384979248, 242.84926414489746, 349.6418023109436)
center= 77.2237578034401 y= 296.24553322792053
scara xy (761.488654715245, 362.0954820991632)
scarxbox (761.488654715245, 362.0954820991632)
box (761.488654715245, 362.0954820991632)
scres[i 1.0
boxes [0.6779075  0.05808482 0.9705161  0.2707161 ]
rect,width=102.06301510334015height=105.33908128738403
(left, right, top, bottom) (27.8807133436203, 129.94372844696045, 244.0467095375061, 349.38579082489014)
center= 78.91222089529037 y= 296.7162501811981
scara xy (759.4765958307415, 361.54758759204395)
scarxbox (759.4765958307415, 361.54758759204395)
box (759.4765958307415, 361.54758759204395)
scres[i 1.0
boxes [0.67572296 0.06540252 0.9715285  0.27939644]
rect,width=102.71708250045776height=106.48998498916626
(left, right, top, bottom) (31.393210887908936, 134.1102933

scres[i 1.0
boxes [0.6738407  0.14084935 0.97221196 0.35744232]
rect,width=103.96462440490723height=107.41365194320679
(left, right, top, bottom) (67.6076889038086, 171.57231330871582, 242.58265256881714, 349.9963045120239)
center= 119.5900011062622 y= 296.28947854042053
scara xy (711.0028746872854, 362.0443316250519)
scarxbox (711.0028746872854, 362.0443316250519)
box (711.0028746872854, 362.0443316250519)
scres[i 1.0
boxes [0.672053   0.14522885 0.9710759  0.3616365 ]
rect,width=103.87566804885864height=107.64824867248535
(left, right, top, bottom) (69.70984697341919, 173.58551502227783, 241.93907260894775, 349.5873212814331)
center= 121.64768099784851 y= 295.76319694519043
scara xy (708.5508382007824, 362.6569010114247)
scarxbox (708.5508382007824, 362.6569010114247)
box (708.5508382007824, 362.6569010114247)
scres[i 1.0
boxes [0.672488   0.15048406 0.9728714  0.36787608]
rect,width=104.34817314147949height=108.13804149627686
(left, right, top, bottom) (72.232346534729, 176.58051967

...

scres[i 1.0
boxes [0.6678335  0.41041407 0.96654683 0.63015276]
rect,width=105.47457218170166height=107.53679752349854
(left, right, top, bottom) (196.99875354766846, 302.4733257293701, 240.42006254196167, 347.9568600654602)
center= 249.7360396385193 y= 294.18846130371094
scara xy (555.9142059590808, 364.48982630150385)
scarxbox (555.9142059590808, 364.48982630150385)
box (555.9142059590808, 364.48982630150385)
scres[i 1.0
boxes [0.6659774  0.41414186 0.9664004  0.6349716 ]
rect,width=105.99828243255615height=108.15226793289185
(left, right, top, bottom) (198.78809452056885, 304.786376953125, 239.75187063217163, 347.9041385650635)
center= 251.78723573684692 y= 293.82800459861755
scara xy (553.4698958918844, 364.9093825707222)
scarxbox (553.4698958918844, 364.9093825707222)
box (553.4698958918844, 364.9093825707222)
scres[i 1.0
boxes [0.6642213  0.4173265  0.96536076 0.6397578 ]
rect,width=106.76702499389648height=108.41021060943604
(left, right, top, bottom) (200.31672477722168, 307.08

scres[i 1.0
boxes [0.6658834  0.5036035  0.96072227 0.72331923]
rect,width=105.46354293823242height=106.14198446273804
(left, right, top, bottom) (241.72968864440918, 347.1932315826416, 239.71803188323975, 345.8600163459778)
center= 294.4614601135254 y= 292.78902411460876
scara xy (502.6171098814413, 366.11871165103)
scarxbox (502.6171098814413, 366.11871165103)
box (502.6171098814413, 366.11871165103)
scres[i 1.0
boxes [0.66664976 0.5117812  0.95856285 0.727081  ]
rect,width=103.34389686584473height=105.0887131690979
(left, right, top, bottom) (245.6549835205078, 348.99888038635254, 239.99391317367554, 345.08262634277344)
center= 297.3269319534302 y= 292.5382697582245
scara xy (499.2024672389083, 366.4105790536008)
scarxbox (499.2024672389083, 366.4105790536008)
box (499.2024672389083, 366.4105790536008)
scres[i 1.0
boxes [0.66527975 0.5140323  0.9581956  0.7314629 ]
rect,width=104.36668395996094height=105.44971704483032
(left, right, top, bottom) (246.73550605773926, 351.102190017700

scres[i 1.0
boxes [0.65850836 0.5906548  0.9564309  0.8086262 ]
rect,width=104.62626457214355height=107.2521185874939
(left, right, top, bottom) (283.5142993927002, 388.14056396484375, 237.06300973892212, 344.315128326416)
center= 335.827431678772 y= 290.68906903266907
scara xy (453.3233047602598, 368.56297002451885)
scarxbox (453.3233047602598, 368.56297002451885)
box (453.3233047602598, 368.56297002451885)
scres[i 1.0
boxes [0.6602425  0.5939767  0.95620793 0.81222314]
rect,width=104.75830078125height=106.54755592346191
(left, right, top, bottom) (285.1088047027588, 389.8671054840088, 237.6872992515564, 344.2348551750183)
center= 337.4879550933838 y= 290.96107721328735
scara xy (451.3445402226029, 368.2463640747765)
scarxbox (451.3445402226029, 368.2463640747765)
box (451.3445402226029, 368.2463640747765)
scres[i 1.0
boxes [0.6602826  0.59721196 0.9577501  0.81750447]
rect,width=105.74040412902832height=107.08828926086426
(left, right, top, bottom) (286.66173934936523, 392.4021434783

scres[i 1.0
boxes [0.6562106  0.67634636 0.9527755  0.889898  ]
rect,width=102.50478744506836height=106.76335573196411
(left, right, top, bottom) (324.64625358581543, 427.1510410308838, 236.23581647872925, 342.99917221069336)
center= 375.8986473083496 y= 289.6174943447113
scara xy (405.57239696162014, 369.8102373715138)
scarxbox (405.57239696162014, 369.8102373715138)
box (405.57239696162014, 369.8102373715138)
scres[i 1.0
boxes [0.6542813 0.677832  0.9520977 0.8946013]
rect,width=104.04925346374512height=107.2139024734497
(left, right, top, bottom) (325.3593635559082, 429.4086170196533, 235.5412745475769, 342.7551770210266)
center= 377.38399028778076 y= 289.14822578430176
scara xy (403.802386380563, 370.356446010956)
scarxbox (403.802386380563, 370.356446010956)
box (403.802386380563, 370.356446010956)
scres[i 1.0
boxes [0.65624356 0.6813822  0.9534784  0.9016201 ]
rect,width=105.71419715881348height=107.00453996658325
(left, right, top, bottom) (327.0634460449219, 432.77764320373535,

scres[i 1.0
boxes [0.6516077  0.75622046 0.9458435  0.9711324 ]
rect,width=103.15773010253906height=105.92489719390869
(left, right, top, bottom) (362.9858207702637, 466.14355087280273, 234.57876920700073, 340.5036664009094)
center= 414.5646858215332 y= 287.5412178039551
scara xy (359.4959700059365, 372.2269349304607)
scarxbox (359.4959700059365, 372.2269349304607)
box (359.4959700059365, 372.2269349304607)
scres[i 1.0
boxes [0.6519838  0.7605181  0.9475008  0.97438556]
rect,width=102.65639305114746height=106.38612985610962
(left, right, top, bottom) (365.0486755371094, 467.70506858825684, 234.7141671180725, 341.10029697418213)
center= 416.3768720626831 y= 287.9072320461273
scara xy (357.33647629566235, 371.80090992452614)
scarxbox (357.33647629566235, 371.80090992452614)
box (357.33647629566235, 371.80090992452614)
scres[i 1.0
boxes [0.6513322  0.7644499  0.9469109  0.98181427]
rect,width=104.33489799499512height=106.40833854675293
(left, right, top, bottom) (366.9359493255615, 471.27

In [7]:
belttestposition=(520.0,364.36,-90,-90,'皮带视频测试点')
belttestendposition=(235.00,364.36,-90,-90,'皮带测试结束点')
import cv2
from io import BytesIO
from base64 import b64encode
cap = cv2.VideoCapture('/home/aadebugergf/aixgf/tzimagedata/beltvideo/belt_output2019052800.mp4')
ret, frame = cap.read()
img_str = cv2.imencode('.jpg', frame)[1].tostring()
import time
start = time.time()
image1=widgets.Image(
    value=img_str,
    format='jpg',
    width=300,
    height=400,
)
display(image1)

button1=widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
bExit=False
ivalue=0
def capturephoto(frame,count):
    
    cv2.imwrite("/home/aadebugergf/aixgf/tzimagedata/beltvision/object{0}.jpg".format(count),frame)

def on_button_clicked(b):
    global bExit
    global ivalue
    print("Button clicked.",bExit)
    bExit=True
    ivalue=10
    print(time.time())
def isExit1():
    global bExit
    return bExit
button1.on_click(on_button_clicked)
display(button1)
def videooutput():
    global ivalue
    count = 0 
    while(True):
        ret, frame = cap.read()
        if frame is None:
            break
        box,newframe=detect_objectsbyframe(frame)
        if box is not None:
            print("box",box)
            b = BytesIO()  
            newframe.save(b, format='JPEG')
            img_str =b.getvalue()
        else:
               img_str = cv2.imencode('.jpg', frame)[1].tostring()

        image1.value=img_str
        end = time.time()
        time.sleep(0.1)
        if ( (end-start)>360):
            break
videooutput()
cap.release()
print("capture end")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

scres[i 0.9774246
boxes [0.67963254 0.02693554 0.95630366 0.2112407 ]
too lower
scres[i 0.96544456
boxes [0.6804999  0.02553177 0.96038383 0.22540708]
too lower
scres[i 0.9830102
boxes [0.68759876 0.03367133 0.9581763  0.2234985 ]
too lower
scres[i 0.99594563
boxes [0.69552076 0.0382865  0.96306086 0.2298642 ]
too lower
scres[i 0.99553347
boxes [0.7005311  0.04276808 0.9721528  0.24249353]
too lower
scres[i 0.99899083
boxes [0.69969773 0.03771948 0.9650264  0.24971317]
rect,width=101.75697147846222height=95.51831245422363
(left, right, top, bottom) (18.105348944664, 119.86232042312622, 251.8911838531494, 347.40949630737305)
center= 68.98383468389511 y= 299.65034008026123
scara xy (771.3077680854631, 358.1324317794112)
scarxbox (771.3077680854631, 358.1324317794112)
box (771.3077680854631, 358.1324317794112)
scres[i 0.9990095
boxes [0.7000518  0.03956232 0.96355534 0.2506218 ]
rect,width=101.30854904651642height=94.86127853393555
(left, right, top, bottom) (18.98991286754608, 120.298461

scres[i 0.99996233
boxes [0.6892655  0.0891896  0.98946625 0.3277204 ]
rect,width=114.4947874546051height=108.07227373123169
(left, right, top, bottom) (42.81100630760193, 157.30579376220703, 248.13557624816895, 356.20784997940063)
center= 100.05840003490448 y= 302.1717131137848
scara xy (734.2777284012267, 355.197660839365)
scarxbox (734.2777284012267, 355.197660839365)
box (734.2777284012267, 355.197660839365)
scres[i 0.9999708
boxes [0.690811   0.09185891 0.9874394  0.33586392]
rect,width=117.1224045753479height=106.78622961044312
(left, right, top, bottom) (44.092276096343994, 161.2146806716919, 248.69195222854614, 355.47818183898926)
center= 102.65347838401794 y= 302.0850670337677
scara xy (731.1853004568321, 355.2985131901763)
scarxbox (731.1853004568321, 355.2985131901763)
box (731.1853004568321, 355.2985131901763)
scres[i 0.9999839
boxes [0.68958527 0.09900799 0.9884663  0.3377065 ]
rect,width=114.57528948783875height=107.5971794128418
(left, right, top, bottom) (47.52383351325

scres[i 0.99999154
boxes [0.6895484  0.17303231 0.9900387  0.41987786]
rect,width=118.48586082458496height=108.17651510238647
(left, right, top, bottom) (83.05551052093506, 201.54137134552002, 248.2374143600464, 356.41392946243286)
center= 142.29844093322754 y= 302.3256719112396
scara xy (683.9423376061993, 355.0184593492533)
scarxbox (683.9423376061993, 355.0184593492533)
box (683.9423376061993, 355.0184593492533)
scres[i 0.9999838
boxes [0.6837107  0.17617066 0.98963624 0.4266155 ]
rect,width=120.21352529525757height=110.13319730758667
(left, right, top, bottom) (84.56191778182983, 204.7754430770874, 246.13584995269775, 356.2690472602844)
center= 144.66868042945862 y= 301.2024486064911
scara xy (681.1178391260064, 356.32584348881954)
scarxbox (681.1178391260064, 356.32584348881954)
box (681.1178391260064, 356.32584348881954)
scres[i 0.99999166
boxes [0.6809405  0.18318355 0.9876087  0.42274094]
rect,width=114.98754501342773height=110.40053844451904
(left, right, top, bottom) (87.9281

...

scres[i 0.9999933
boxes [0.6795988  0.43833363 0.9852287  0.6812535 ]
rect,width=116.60153388977051height=110.02676725387573
(left, right, top, bottom) (210.40014266967773, 327.00167655944824, 244.65557098388672, 354.68233823776245)
center= 268.700909614563 y= 299.6689546108246
scara xy (533.3146979728342, 358.11076525778446)
scarxbox (533.3146979728342, 358.11076525778446)
box (533.3146979728342, 358.11076525778446)
scres[i 0.99999356
boxes [0.67449504 0.4411653  0.9850806  0.6884306 ]
rect,width=118.68734836578369height=111.81080102920532
(left, right, top, bottom) (211.75934314727783, 330.4466915130615, 242.81821489334106, 354.6290159225464)
center= 271.1030173301697 y= 298.7236154079437
scara xy (530.4522236942732, 359.2110998707965)
scarxbox (530.4522236942732, 359.2110998707965)
box (530.4522236942732, 359.2110998707965)
scres[i 0.99999356
boxes [0.6786228  0.44565922 0.98475975 0.690897  ]
rect,width=117.7141284942627height=110.20930767059326
(left, right, top, bottom) (213.9164

scres[i 0.9999728
boxes [0.6707042 0.535097  0.978885  0.7803772]
rect,width=117.73449897766113height=110.9450912475586
(left, right, top, bottom) (256.84656143188477, 374.5810604095459, 241.45350694656372, 352.3985981941223)
center= 315.71381092071533 y= 296.926052570343
scara xy (477.2917228684901, 361.303386537052)
scarxbox (477.2917228684901, 361.303386537052)
box (477.2917228684901, 361.303386537052)
scres[i 0.9999825
boxes [0.66825616 0.53873944 0.9775088  0.7824538 ]
rect,width=116.98287963867188height=111.33094310760498
(left, right, top, bottom) (258.59493255615234, 375.5778121948242, 240.5722188949585, 351.9031620025635)
center= 317.0863723754883 y= 296.237690448761
scara xy (475.6561085142689, 362.10461076043646)
scarxbox (475.6561085142689, 362.10461076043646)
box (475.6561085142689, 362.10461076043646)
scres[i 0.9999809
boxes [0.67323947 0.53783035 0.9771168  0.7835484 ]
rect,width=117.94466972351074height=109.39584732055664
(left, right, top, bottom) (258.1585693359375, 3

scres[i 0.9999747
boxes [0.66907185 0.61815625 0.9713162  0.86159223]
rect,width=116.84926986694336height=108.80797147750854
(left, right, top, bottom) (296.7150020599365, 413.5642719268799, 240.86586713790894, 349.6738386154175)
center= 355.1396369934082 y= 295.2698528766632
scara xy (430.30989423904487, 363.2311325180314)
scarxbox (430.30989423904487, 363.2311325180314)
box (430.30989423904487, 363.2311325180314)
scres[i 0.99996865
boxes [0.6695362  0.6239195  0.9716428  0.86590266]
rect,width=116.15192413330078height=108.75838279724121
(left, right, top, bottom) (299.4813537597656, 415.6332778930664, 241.03302240371704, 349.79140520095825)
center= 357.557315826416 y= 295.41221380233765
scara xy (427.4288646214541, 363.0654304572445)
scarxbox (427.4288646214541, 363.0654304572445)
box (427.4288646214541, 363.0654304572445)
scres[i 0.99998164
boxes [0.6704786  0.62856    0.97309977 0.87387645]
rect,width=117.75189399719238height=108.94362688064575
(left, right, top, bottom) (301.70880

scres[i 0.99999666
boxes [0.67320484 0.7105195  0.9753239  0.9412903 ]
rect,width=110.7699966430664height=108.76286745071411
(left, right, top, bottom) (341.0493564605713, 451.8193531036377, 242.35374212265015, 351.11660957336426)
center= 396.4343547821045 y= 296.7351758480072
scara xy (381.10099875838466, 361.5255589210644)
scarxbox (381.10099875838466, 361.5255589210644)
box (381.10099875838466, 361.5255589210644)
scres[i 0.9999933
boxes [0.6784999  0.71330804 0.96775717 0.9370139 ]
rect,width=107.37882614135742height=104.13262367248535
(left, right, top, bottom) (342.3878574371338, 449.7666835784912, 244.25995588302612, 348.3925795555115)
center= 396.0772705078125 y= 296.3262677192688
scara xy (381.5265186232094, 362.0015105860558)
scarxbox (381.5265186232094, 362.0015105860558)
box (381.5265186232094, 362.0015105860558)
scres[i 0.9999962
boxes [0.6759364 0.7126518 0.966477  0.9443945]
rect,width=111.23651504516602height=104.59460735321045
(left, right, top, bottom) (342.07285881042

scres[i 0.99999785
boxes [0.67843896 0.84406525 0.95303845 0.9963778 ]
too lower
scres[i 0.99999917
boxes [0.6805425  0.85086936 0.95106256 0.9962612 ]
too lower
scres[i 0.99999976
boxes [0.672263   0.85296834 0.9481519  0.9980695 ]
too lower
scres[i 0.9999993
boxes [0.6785097 0.8582511 0.941678  1.       ]
too lower
scres[i 0.99999976
boxes [0.68020266 0.8592627  0.9430968  0.9967981 ]
too lower
scres[i 0.99999964
boxes [0.67895085 0.8658056  0.94172305 0.9965628 ]
too lower
scres[i 0.9999993
boxes [0.68275034 0.8658359  0.94921875 0.9994457 ]
too lower
scres[i 0.99999857
boxes [0.68010014 0.87276953 0.9488554  0.99853325]
too lower
scres[i 0.9999987
boxes [0.6909555  0.8810516  0.94429517 1.        ]
too lower
scres[i 0.99999917
boxes [0.6893861  0.88544095 0.942144   0.9987494 ]
too lower
scres[i 0.9999931
boxes [0.6916599 0.8926574 0.9315917 1.       ]
too lower
scres[i 0.9999957
boxes [0.6926271 0.8913191 0.9320422 1.       ]
too lower
scres[i 0.99999845
boxes [0.69234043 0.897022